In [8]:
import cv2
import numpy as np
import os
from skimage.measure import shannon_entropy
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Paths to your healthy and diseased datasets
healthy_path = r"E:\OneDrive\Desktop\FINAL_DIP_DIP\Tomato___healthy"
diseased_path = r"E:\OneDrive\Desktop\FINAL_DIP_DIP\Peach___Bacterial_spot"

# Paths to save processed images
output_folder = "processed_images"
os.makedirs(output_folder, exist_ok=True)
steps = ["Grayscale", "SpotDetection", "HOG", "LBP"]

# Create subfolders for each processing step
for step in steps:
    os.makedirs(os.path.join(output_folder, step), exist_ok=True)

# Load images from dataset
def load_images(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            filenames.append(filename)
    return images[:100], filenames[:100]  # Limit to first 100 images

# Load datasets with a limit of 100 images each
healthy_images, healthy_filenames = load_images(healthy_path)
diseased_images, diseased_filenames = load_images(diseased_path)

# Utility function to save images
def save_image(image, step, filename):
    """Saves processed image to the appropriate step folder."""
    if not filename.endswith(".png"):
        filename += ".png"  # Add a default extension if not present
    cv2.imwrite(os.path.join(output_folder, step, filename), image)

# Processing functions with saving
def to_grayscale(image, filename):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    save_image(gray, "Grayscale", filename)
    return gray

def spot_detection(image, filename):
    gray = to_grayscale(image, filename)
    _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    save_image(thresh, "SpotDetection", filename)
    return len(contours)

def save_hog(image, filename):
    gray = to_grayscale(image, filename)
    hog_features, hog_image = hog(gray, orientations=8, pixels_per_cell=(16, 16),
                                  cells_per_block=(1, 1), visualize=True)
    hog_image = (hog_image * 255).astype(np.uint8)  # Scale hog_image to [0, 255]
    save_image(hog_image, "HOG", filename)
    return hog_features

def save_lbp(image, filename):
    gray = to_grayscale(image, filename)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    lbp_image = (lbp * 255 / lbp.max()).astype(np.uint8)  # Scale LBP image to [0, 255]
    save_image(lbp_image, "LBP", filename)
    return lbp

# Function to extract features from an image
def extract_features(image, filename):
    features = {}
    # Spot Detection feature
    features['spot_count'] = spot_detection(image, filename)
    
    # HSV features
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    features['hue_mean'] = np.mean(h)
    features['hue_std'] = np.std(h)
    features['saturation_mean'] = np.mean(s)
    features['saturation_std'] = np.std(s)
    features['value_mean'] = np.mean(v)
    features['value_std'] = np.std(v)
    
    # Shannon entropy for texture
    gray = to_grayscale(image, filename)
    features['entropy'] = shannon_entropy(gray)
    
    # Save and extract HOG features
    features['hog_features'] = save_hog(image, filename)
    
    # Save and extract LBP features
    features['lbp'] = save_lbp(image, filename)
    
    # Color Histogram (HSV histograms)
    features['h_hist'] = np.histogram(h, bins=8, range=(0, 255))[0].mean()
    features['s_hist'] = np.histogram(s, bins=8, range=(0, 255))[0].mean()
    features['v_hist'] = np.histogram(v, bins=8, range=(0, 255))[0].mean()
    
    return features

# Collect features and labels
data = []
labels = []

# For Healthy images
for img, fname in zip(healthy_images, healthy_filenames):
    features = extract_features(img, fname)
    features['label'] = 0  # Healthy
    data.append(features)

# For Diseased images
for img, fname in zip(diseased_images, diseased_filenames):
    features = extract_features(img, fname)
    features['label'] = 1  # Diseased
    data.append(features)

# Convert data to DataFrame for ease of use
df = pd.DataFrame(data)

# Separate features and labels
X = df.drop(columns=['label'])
y = df['label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate on the test set
y_pred = clf.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))

# Function to classify a new image using the trained model
def classify_new_image(image, filename, classifier):
    features = extract_features(image, filename)
    feature_values = np.array([list(features.values())])
    prediction = classifier.predict(feature_values)
    classification = "Healthy" if prediction[0] == 0 else "Diseased"
    print(f"{filename}: {classification}")
    return classification

# Example usage with new images (assuming you have some images to test)
print("\nClassifying new images:")
for img, fname in zip(healthy_images[:5], healthy_filenames[:5]):  # Testing with a few healthy images
    classify_new_image(img, fname, clf)

for img, fname in zip(diseased_images[:5], diseased_filenames[:5]):  # Testing with a few diseased images
    classify_new_image(img, fname, clf)

# Specify the path of the new image you want to classify
new_image_path = r"E:\OneDrive\Desktop\FINAL_DIP_DIP\Tomato___healthy\87f08e15-5b2d-4d01-b83b-fd958eba02d4___GH_HL Leaf 434.2.JPG"
new_image = cv2.imread(new_image_path)

# Check if the image was loaded successfully
if new_image is not None:
    # Classify the new image
    new_image_filename = "new_image"  # Use any name for the filename reference
    classification = classify_new_image(new_image, new_image_filename, clf)
    print(f"Classification of the new image: {classification}")
else:
    print("Failed to load the new image.")


ValueError: setting an array element with a sequence.